## Prueba Modelo Ridge con / sin intercepto

In [1]:
parametro = False

In [2]:
from sklearn.linear_model import Ridge
import pandas as pd
import numpy.linalg as lg
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
path = "C:/Users/Alber/OneDrive/Documentos/USTA 20242/INTELIGENCIA ARTIFICIAL/CORTE 1/SEMANA 2/"

In [4]:
def load_data_python():
    dataset = pd.read_csv(path+"base.csv",header=0,sep=";",decimal=",")
    prueba = pd.read_csv(path+"prueba.csv",header=0,sep=";",decimal=",")
    covariables = [x for x in dataset.columns if x not in ["y"]]
    X_train, y_train = dataset.get(covariables), dataset.get(["y"])
    X_test, y_test = prueba.get(covariables), prueba.get(["y"])
    return X_train, X_test, y_train, y_test

In [5]:
def model(lambda_par):
    X_train, X_test, y_train, y_test = load_data_python()
    modelo = Ridge(alpha = lambda_par,fit_intercept=parametro)
    modelo.fit(X_train, y_train )
    if parametro==True:
        r1 = [float(modelo.intercept_)] + [x for x in modelo.coef_[0]]
    else:
        r1 = modelo.coef_
    return r1

In [6]:
cf_funcion = model(lambda_par = 0.5)
if parametro==False:
    cf_funcion = [x for x in cf_funcion[0]]
cf_funcion

[0.5197328862342071,
 1.2867267294505025,
 -0.10280247022215629,
 -101.14044038412786,
 698145.5505560889,
 -1466.4455056699665,
 56.07873979389588,
 -54.880233079763364,
 1.1953814039799548,
 1.2217046133487257,
 1.2446433477325352,
 1.276612176148848,
 1.3148251777363393,
 1.3589140705457885,
 1.4208508560401165,
 1.4814646728951766,
 1.573685125042088,
 1.7261092596938448,
 1.0257476790372488,
 1.850752457665116,
 1.0526399395159984,
 1.0673038444123986,
 1.0871982244268261,
 1.1069983170880342,
 1.1251563129697328,
 1.1500826707343716,
 1.165868335889749]

$\beta_{Ridge} = (X^tX + \lambda I)^{-1} X^{t} y$

In [7]:
def coefficientes_pedal(lambda_par):
    X_train, X_test, y_train, y_test = load_data_python()
    if parametro==True:
        covariables = list(X_train.columns)
        X_barra = np.mean(X_train,axis = 0)
        y_barra = np.mean(y_train,axis = 0)
        X_train_c = X_train - X_barra
        y_train_c = y_train - y_barra
        X_train_c["intercepto"] = 1
        X_train_c = X_train_c.get(["intercepto"]+covariables)
        X_train = X_train_c.to_numpy()
        y_train = y_train_c.to_numpy()
    else:
        X_train = X_train.to_numpy()
        y_train = y_train.to_numpy()
    p = X_train.shape[1]
    Id = np.eye(p)
    prod1 = lg.inv(np.matmul(X_train.transpose(),X_train) + lambda_par * Id)
    prod2 = np.matmul(X_train.transpose(),y_train)
    coef = np.matmul(prod1,prod2)

    if parametro==True:
        coef[0] = y_barra - np.dot(X_barra, coef[1:])

    return coef

In [8]:
cf_pedal = coefficientes_pedal(lambda_par = 0.5)
cf_pedal = cf_pedal.transpose()
cf_pedal = [x for x in cf_pedal[0]]
cf_pedal

[0.5197329615834718,
 1.2867266873807601,
 -0.10280238442092227,
 -101.1404400341977,
 698145.5505760629,
 -1466.4453417289187,
 56.07874645060883,
 -54.880239749647444,
 1.1953814039649626,
 1.221704613348447,
 1.2446433477182233,
 1.276612176116525,
 1.3148251777258655,
 1.3589140705450207,
 1.420850856034087,
 1.4814646729102232,
 1.573685124976802,
 1.7261092595372955,
 1.0257476790376663,
 1.850752457561065,
 1.052639939503905,
 1.0673038444136846,
 1.087198224411904,
 1.1069983170791402,
 1.1251563129700366,
 1.150082670698998,
 1.1658683358704591]

In [9]:
for i,j in zip(cf_funcion,cf_pedal):
    print(np.round(i,2)==np.round(j,2))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


## Prueba performance del modelo

In [57]:
Train_Data = False

In [58]:
def umbral(ytrue,yhat,u = 0.07):
    return 100 * np.mean(np.abs((ytrue-yhat)/ytrue)<=u)

In [59]:
def model_predictions(lambda_par,parametro):
    X_train, X_test, y_train, y_test = load_data_python()
    if parametro==True:
        modelo = Ridge(alpha = lambda_par,fit_intercept=False)
        modelo.fit(X_train,y_train)
        modelo2 = Ridge(alpha = lambda_par,fit_intercept=True)
        modelo2.fit(X_train,y_train)
        yhat_train1 = modelo.predict(X_train)
        yhat_test1 = modelo.predict(X_test)
        yhat_train2 = modelo2.predict(X_train)
        yhat_test2 = modelo2.predict(X_test)
    else:
        modelo = Ridge(alpha = lambda_par,fit_intercept=False)
        modelo.fit(X_test,y_test)
        modelo2 = Ridge(alpha = lambda_par,fit_intercept=True)
        modelo2.fit(X_test,y_test)
        yhat_train1 = modelo.predict(X_test)
        yhat_test1 = modelo.predict(X_train)
        yhat_train2 = modelo2.predict(X_test)
        yhat_test2 = modelo2.predict(X_train)
    return yhat_train1, yhat_test1, yhat_train2, yhat_test2

In [60]:
yhat_train1, yhat_test1, yhat_train2, yhat_test2 = model_predictions(0.5,parametro=Train_Data)
X_train, X_test, y_train, y_test = load_data_python()

In [61]:
if Train_Data==True:
    a = umbral(y_test,yhat_test1)>umbral(y_test,yhat_test2)
else:
    a = umbral(y_train,yhat_test1)>umbral(y_train,yhat_test2)

if a==True:
    b = f"El mejor modelo es sin intercepto"
else:
    b = f"El mejor modelo es con intercepto"

In [62]:
if (Train_Data==True) & (a==True):
    u1 = umbral(y_train,yhat_train1) 
    u2 = umbral(y_test,yhat_test1) 
elif (Train_Data==True) & (a==False):
    u1 = umbral(y_train,yhat_train2) 
    u2 = umbral(y_test,yhat_test2) 
elif (Train_Data==False) & (a==False):
    u1 = umbral(y_test,yhat_train2)
    u2 = umbral(y_train,yhat_test2)
else:
    u1 = umbral(y_test,yhat_train1)
    u2 = umbral(y_train,yhat_test1)

## No hay underfitting
Si más del 80% de la información está bien predicha

## No hay overfitting
Si no hay más de 5 puntos de diferencia entre error de entrenamiento y test

In [63]:
valores = [u1,u2,-np.abs(u1-u2)]
comparacion = [80,80,-5]
for i,j in zip(valores,comparacion):
    print(i>=j)

True
True
True


In [64]:
print(b)

El mejor modelo es con intercepto
